In [3]:
from langchain import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from fastapi import FastAPI, Request, Form, Response
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.encoders import jsonable_encoder
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
import os
import json

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.llms import Ollama

In [21]:
!pip install openai==0.28

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
system_msg = "I am dedicated to offering safe, respectful, and honest assistance, prioritizing helpfulness, positivity, and accuracy in my responses."""
chat = f"""
<|im_start|>system
{system_msg}<|im_end|>
<|im_start|>user
{prompt_template}<|im_end|>
<|im_start|>assistant
""".strip()
print(chat)

<|im_start|>system
I am dedicated to offering safe, respectful, and honest assistance, prioritizing helpfulness, positivity, and accuracy in my responses.<|im_end|>
<|im_start|>user
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
<|im_end|>
<|im_start|>assistant


In [10]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
system_message = "you are a helpful medical assistant."


In [7]:
def format_prompt(prompt):
    # if args.shots > 0:
    #     prompt = prompt[:-1]
    # if "orca" in args.checkpoint_name:
    #     system_msg = "You are an AI assistant who helps people find information."
    #     return f"<|im_start|> system\n{system_msg}<|im_end|>\n<|im_start|> question\n{prompt}<|im_end|>\n<|im_start|> answer\n"
    system_msg = "You are a helpful, respectful and honest assistant." + \
    "Always answer as helpfully as possible, while being safe." + \
    "Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content." + \
    "Please ensure that your responses are socially unbiased and positive in nature.\n\n" + \
    "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct." + \
    "If you don't know the answer to a question, please don't share false information."""
    return f"<|im_start|> system\n{system_msg}<|im_end|>\n <|im_start|> user\n{prompt}<|im_end|>\n <|im_start|> assistant\n"

In [8]:
prompt_template = format_prompt(prompt_template)

In [1]:
import openai, os
from langchain.llms import OpenAI
openai.api_key = "EMPTY"
openai.api_base = "http://34.90.142.9:8000/v1"
os.environ['OPENAI_API_KEY'] = openai.api_key
llm = OpenAI(model="TheBloke/meditron-7B-AWQ")

In [23]:
llm("what is cancer?")

'\nCancer is a disease of the cells in which the cells grow out of control. Cancer cells may invade to other tissues and organs of the body beyond the primary site of the tumor.\n\n## Different types of cancer\nCancer has different types according to their appearance, their origin and their development.\n- Carcinomas are tumors originating from epithelial tissues and are the most common type of cancer in humans. - Sarcomas are tumors originating from mesenchymal tissues, and are the second most common type of cancer in humans.\n- Leukemias are malignant tumors of the blood, of the hematopoietic system: lymphatic system and bone marrow. - Lymphomas are malignant tumors of the lymphatic system, they are originated from lymphoid tissue. - Central nervous system tumors are tumors originating from the central nervous system, they are divided into primary and secondary tumors.\n\n## Risk factors\nThe risk factors of cancer are divided into two categories: lifestyle risk factors and genetic r

In [25]:

embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

url = "http://localhost:6333"

client = QdrantClient(
    url=url, prefer_grpc=False
)

db = Qdrant(client=client, embeddings=embeddings, collection_name="vector_db")

prompt = PromptTemplate(template=chat, input_variables=['context', 'question'])

retriever = db.as_retriever(search_kwargs={"k":1})


In [26]:
query = "what is cancer?"
chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)
print(query)
response = qa(query)
print(response)
answer = response['result']
source_document = response['source_documents'][0].page_content
doc = response['source_documents'][0].metadata['source']
# response_data = jsonable_encoder(json.dumps({"answer": answer, "source_document": source_document, "doc": doc}))

# res = Response(response_data)

what is cancer?


> Entering new RetrievalQA chain...


OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacty of 14.58 GiB of which 69.38 MiB is free. Including non-PyTorch memory, this process has 1.15 GiB memory in use. Process 1014849 has 13.35 GiB memory in use. Of the allocated memory 426.85 MiB is allocated by PyTorch, and 45.15 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:

local_llm = "meditron-7b.Q4_K_M.gguf"

config = {
'max_new_tokens': 512,
'context_length': 1024,
'repetition_penalty': 1.1,
'temperature': 0.1,
'top_k': 50,
'top_p': 0.9,
'stream': True,
'threads': int(os.cpu_count() / 2)
}
# try:
   
#    llm = CTransformers(
#    model=local_llm,
#    model_type="llama",
#    lib="avx2",
#    **config
#    )
# except Exception as e:
#    print(e)

In [3]:

config

{'max_new_tokens': 512,
 'context_length': 1024,
 'repetition_penalty': 1.1,
 'temperature': 0.1,
 'top_k': 50,
 'top_p': 0.9,
 'stream': True,
 'threads': 4}